In [ ]:
import numpy as np

from pathlib import Path
import sys

ixdat_source_path = Path.home() / "git/ixdat/src"
sys.path.append(str(ixdat_source_path))

import ixdat

In [ ]:
data_directory = Path.home() / "Dropbox/ixdat_resources/jupyter_notebooks/21A29_ixdat_demo/demo_data"
[f.name for f in data_directory.iterdir()]

In [ ]:
from ixdat.techniques import CyclicVoltammagram

cv_meas = CyclicVoltammagram.read(
    data_directory / "01_demo_03_CVA_C01.mpt", reader="biologic"  # technique as option kwarg.
)   # if it's a TechniqueMeasureemtn instead of Measurement, it should return obj of TechniqueMeasurement.

cv_meas

In [ ]:
cv_meas.plot_measurement(tspan=[13000, 14000])

In [ ]:
co_strip = cv_meas.cut(tspan=[13300, 13900])
co_strip.tstamp += co_strip.t[0]

In [ ]:
co_strip.calibrate(A_el=0.196, RE_vs_RHE=0.715)
co_strip.plot_measurement()

In [ ]:
co_strip.redefine_cycle(start_potential=0.35, redox=False)
ax = co_strip[1].plot(color="r")
co_strip[2].plot(color="k", ax=ax)

In [ ]:
stripping = co_strip[1].select_sweep(vspan=[0.6, 1.0])
base = co_strip[2].select_sweep(vspan=[0.6, 1.0])
ax = stripping.plot(color="r")
base.plot(color="k", ax=ax)

In [ ]:
t_strip, I_strip = stripping.grab("raw_current")
t_base, I_base = base.grab("raw_current")

dQ = np.trapz(I_strip, t_strip) - np.trapz(I_base, t_base)

from ixdat.constants import Faraday

dn = dQ / (Faraday * 2)

print(f"charge passed = {dQ*1e3} mC, corresponding to {dn*1e9} nmol of CO oxidized")